In [158]:
# This notebook runs enriches and cleans the email metadata. Steps:
# 1. extract emails from paper metadata
# 2. enrich based on doi lookup for biorxiv repository
# 3. enrich based on doi lookup for PMC papers
# 4. match addresses to authors based on name matching
# 5. resolve conflicts where many people match one email
# 6. clean email addresses based on several observed pathologies from metadata
# 7. store output.

# code by Ben Jones (ben.jones@uta.edu) for the CoronaWhy project.


In [159]:
import pylab
import matplotlib
import pandas as pd
import numpy as np
from collections import defaultdict
import copy
import pickle
import os
import json
import re
import requests
%matplotlib inline

In [160]:
# Metadata from the CORD release
meta = pd.read_csv("metadata.csv")

# Paths to the papers (loop over these)
Paths=["./CORD-19-research-challenge/noncomm_use_subset/noncomm_use_subset/","./CORD-19-research-challenge/comm_use_subset/comm_use_subset/","./CORD-19-research-challenge/biorxiv_medrxiv/biorxiv_medrxiv/","./CORD-19-research-challenge/custom_license/custom_license/"]

In [161]:
# Get author information from papers
#  This info lives in the metadata block in each file.

AuthorFirst=[]
AuthorLast=[]
AuthorEmail=[]
AuthorFull=[]
AuthorMiddle=[]
AuthorSuffix=[]
PaperIDs=[]
for Path in Paths:
    print(Path)

    Files=os.listdir(Path)
    for p in Files:

        readfile=open(Path+p,'r')
        paper=json.load(readfile)
        for author in paper['metadata']['authors']:
            AuthorFirst.append(author['first'])
            AuthorLast.append(author['last'])
            AuthorEmail.append(author['email'])
            AuthorMiddle.append(re.sub('\W+','', str(author['middle'])))
            AuthorSuffix.append(author['suffix'])
            AuthorFull.append(AuthorLast[-1] + ", " + AuthorFirst[-1] + " " + AuthorMiddle[-1])
            PaperIDs.append(p[:-5])
authors=pd.DataFrame({'sha':PaperIDs,'first': AuthorFirst,'middle': AuthorMiddle,'last': AuthorLast,'suffix': AuthorSuffix,'email': AuthorEmail,'full': AuthorFull})
authors=authors.replace('',np.NaN)
authors.to_csv("AuthorsFromMetadata.csv")


./CORD-19-research-challenge/noncomm_use_subset/noncomm_use_subset/
./CORD-19-research-challenge/comm_use_subset/comm_use_subset/
./CORD-19-research-challenge/biorxiv_medrxiv/biorxiv_medrxiv/
./CORD-19-research-challenge/custom_license/custom_license/


In [163]:
# Enrich from Biorxiv / Medrxiv websites
# The common format here means we can be very specific on extraction syntax

Path="./CORD-19-research-challenge/biorxiv_medrxiv/biorxiv_medrxiv/"

Files=os.listdir(Path)
TheMatch=0
FileWithFullText=[]
DOI=[]
for f in Files:
    matches=meta[meta.sha==f.split('.')[0]]
    if(len(matches)>0):
        TheMatch=matches.iloc[0]
        if(TheMatch.has_full_text):
            FileWithFullText.append(f.split('.')[0])
            DOI.append(TheMatch.doi)

def PullEmailsBiorxiv(text):
    text=str(r.content)
    if("mailto" in text): 
        pos=text.find("mailto:") 
        Corresp=text[pos:-1].split(":")[1].split('"')[0]
    else:
        Corresp=""
    Auth=""
    while(True):
        if("citation_author_email" in text):
            pos=text.find("citation_author_email")
            Auth=Auth+str(text[pos:-1].split('content="')[1].split('"')[0])+";"
            text=text[pos+50:]
        else:
            break
    return(Corresp,Auth[:-1])

count=0
EmailsCollected={}
SaveCSV=[]
OutFile=open("./BiorxivData.csv","w")
for d in range(0,len(DOI)):
    addr="https://doi.org/"+DOI[d]
    r = requests.get(addr)
    text=r.content
    # We'll only grab the full list, and forget about corredponding for now
    EmailsCollected[FileWithFullText[d]]=PullEmailsBiorxiv(text)[1]
    count=count+1
    if(count%100==0):
        print("Processed "+str(count)+" of " + str(len(DOI)) + " records")
        # print(addr)
        # print(FileWithFullText[d],EmailsCollected[FileWithFullText[d]])


Processed 100 of 1053 records
Processed 200 of 1053 records
Processed 300 of 1053 records
Processed 400 of 1053 records
Processed 500 of 1053 records
Processed 600 of 1053 records
Processed 700 of 1053 records
Processed 800 of 1053 records
Processed 900 of 1053 records
Processed 1000 of 1053 records


In [164]:
PCMPapers=meta[(meta.source_x=='PMC') & meta.notna().sha &meta.notna().doi & meta.has_full_text]

In [165]:
PCMPapers.doi.values

array(['10.1186/1471-2458-3-5', '10.1186/gb-2003-4-5-213',
       '10.1186/1471-2350-4-9', ..., '10.1016/j.celrep.2015.05.012',
       '10.1016/j.ijppaw.2014.06.003', '10.1016/j.onehlt.2017.03.001'],
      dtype=object)

In [166]:
# Enrich from the PCM papers. Here was only reliably
#  extract one email per paper, for now.

def PullEmailPCM(text):
    Email=""
    try:
        if("mailto" in text): 
            pos=text.find("mailto:") 
            Email = text[pos:-1].split(":")[1].split('"')[0]
            if(("info" in Email) or ("webmaster" in Email) or ("biomedcentral" in Email) or (len(Email)>40)or ("editor" in Email)or ("journal" in Email)):
                Email=""
    except:
        return("")
    return(Email)

#Loop through dois pulling emails
count=0
for p in PCMPapers.index:
    addr="https://doi.org/"+PCMPapers.doi[p]
    try:
        r = requests.get(addr)
    except:
        continue
        
    text=r.content
    EmailsCollected[PCMPapers.sha[p]]=PullEmailPCM(str(text))
    count=count+1
    if(count%100==0):
        print("Processed "+str(count)+" of " + str(len(PCMPapers.index)) + " records")
    #    print(addr)
    #    print(PCMPapers.sha[p],EmailsCollected[PCMPapers.sha[p]])


Processed 100 of 14729 records
Processed 200 of 14729 records
Processed 300 of 14729 records
Processed 400 of 14729 records
Processed 500 of 14729 records
Processed 600 of 14729 records
Processed 700 of 14729 records
Processed 800 of 14729 records
Processed 900 of 14729 records
Processed 1000 of 14729 records
Processed 1100 of 14729 records
Processed 1200 of 14729 records
Processed 1300 of 14729 records
Processed 1400 of 14729 records
Processed 1500 of 14729 records
Processed 1600 of 14729 records
Processed 1700 of 14729 records
Processed 1800 of 14729 records
Processed 1900 of 14729 records
Processed 2000 of 14729 records
Processed 2100 of 14729 records
Processed 2200 of 14729 records
Processed 2300 of 14729 records
Processed 2400 of 14729 records
Processed 2500 of 14729 records
Processed 2600 of 14729 records
Processed 2700 of 14729 records
Processed 2800 of 14729 records
Processed 2900 of 14729 records
Processed 3000 of 14729 records
Processed 3100 of 14729 records
Processed 3200 of

In [167]:
# Now we need to run through matching emails to names



#We'll distinguish between 4 strengths of match:
# 4 - only one author and only one email exists
# 3 - match both first and last name in email address
# 2 - match last name
# 1 - match first name
# in case of conflicts, the highest stength option wins.
meta=meta[pd.notna(meta.sha) & meta.has_full_text]
strengths=np.zeros(len(authors.index))
authors.insert(4,'strengths',strengths)

for a in authors.index:
    record=authors.loc[a]
    paper=record.sha
    if(pd.isna(record.email) and (paper in EmailsCollected.keys()) and EmailsCollected[paper]!=''):  # only try to fill a gap if there is a gap to fill

        emailaddresses=EmailsCollected[paper].split(";")
        # use these to count matches
        matchesboth=-1
        matchesfirst=-1
        matcheslast=-1
        match=-1
        
        first=str(record['first']).lower()
        last=str(record['last']).lower()
       
        for ei in range(0,len(emailaddresses)):
            e=emailaddresses[ei].lower()
            
            # matching first and last name
            if((first in e) and ( last in e )and (len(last)>1)and (len(first)>1)):
                if(matchesboth>0):
                    print("   Uh oh... two double matches!")
                    print("   they are " + emailaddresses[ei]+", "+emailaddresses[matchesboth])
                matchesboth=ei
            # matching surname only
            elif ((last in e )and (len(last)>2) and pd.notna(record['last'])):
                if(matchesboth>0):
                    continue
                matcheslast=ei
            # matching first name only
            elif ((first in e ) and (len(first)>2) and pd.notna(record['first'])):
                if((matchesboth>0) or(matcheslast>0)):
                    continue
                matchesfirst=ei
                
            
            # find strongest match
            strength=0

            if((len(emailaddresses)==1) and (sum(authors.sha==paper)==1)):
                match=0
                strength=4 # unique combination
            elif(matchesboth>-1):
                match=matchesboth
                strength=3 # first and last name
            elif(matcheslast>-1):
                match=matcheslast
                strength=2 # last only
            elif(matchesfirst>-1):
                match=matchesfirst
                strength=1 # first only
                
                
        #If weg got a match, store it in the authors table
        if(match>-1):
            print(strength, authors.sha.loc[a], authors['first'].loc[a], authors['last'].loc[a], emailaddresses[match])
            authors.email.loc[a]=emailaddresses[match]
            authors.strengths.loc[a]=strength

2 01aa247284f0baf43293d8ea335b70781365a985 Heungsup Sung sung@amc.seoul.kr


/usr/local/lib/python3.7/site-packages/pandas/core/indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


1 041adbe00a7ca71fcbc38ff4406297ce86e93a6a Rossetto nan erossetto@usp.br
2 054831a6a11886b5ad44cac71d17e367721a2b5b Julian Hiscox j.a.hiscox@leeds.ac.uk
2 0612bb6d157e8cd266943d84daf110c46a56aa23 Sooyeon Suh alysuh@sungshin.ac.kr
1 07241235ee553a5b165ed19b2091101128eca805 Helen O&apos;shea helen.oshea@cit.ie
1 0aa3eda0fb09bf9f55864fc2b9157b9317f62323 Robson Bruniera-Oliveira robson.bruniera@gmail.com
2 0dc8d11784da63b899dbb2b404be4efd330e4ac3 Dongming Zhou dmzhou@sibs.ac.cn
2 0e78935692851b0466f5a9234adb024e7735c8a3 nan Chua chuakb@tll.org.sg
4 0f6091029a2cc8ddd6a5c6ee18de7478b73daab2 Emilio Espínola emilioespinola@hotmail.com
2 104e60eede9e4016c243f20f4541a33852ea3d1c Jin-Hwa Moon jinhwamoon@hanyang.ac.kr
4 105b993dee9f9e44637daa2d43e66f3ba3039fbf Esteban Domingo edomingo@cbm.uam.es
2 107ff91ec95dea35a80145aebbe5322c08c57e5e N Bellei nbellei@uol.com.br
2 11ebffa99e4b093c6304549246b534983012f909 Ji-Eun Ban jeban@naver.com
2 129fe5d1e963bc655146edf42a382819deca0ce6 Adekunle Adeoti Kadeo

2 92aca69f518fc394bd00b525898264893d9550af Young Jeon yjjeon@cnu.ac.kr
2 93f3546142cf477c77ef113ed4ff98a881a2c8c1 Piper Treuting treuting@uw.edu
4 953ce6c11451011ff271b730ac3d2bdadda75830 Ock-Joo Kim ockjoo.kim@gmail.com
2 9547b74fb56cd1288d9af6facbd337148cf45895 Matos Soeiro alexandre.soeiro@bol.com.br
2 9665e16e8fd0e3198298f64f70cc9180d1cea36a Biswendu Goswami bisgoswami@gmail.com
3 972a4e182f215c25fe41c9e0716a2f14ffd2ed9c Jianguo Xu xujianguo@icdc.cn
3 9b2f83db4c307ae17c146bb6625ddb01474bbd9a Rong Kuan kuanrong.chan@duke-nus.edu.sg
2 9b2f83db4c307ae17c146bb6625ddb01474bbd9a nan Chan kuanrong.chan@duke-nus.edu.sg
2 9b2f83db4c307ae17c146bb6625ddb01474bbd9a Eugenia Ong kuanrong.chan@duke-nus.edu.sg
2 9f7ffa3a1bbf4a6ad9e71f2377fb45564589d415 Ji-Youl Jung jungjy1982@jejunu.ac.kr
2 a2add35b84a53cbad3301093f4191d5edadf1d0e Ki Hahm hahmkb@cha.ac.kr
4 a70f7904950bba73ba43df71226c57344b28aa33 São Paulo tterezinha@uol.com.br
4 aad248c894ac1b35e2adad9e23c9cf40dc3c20aa Margaret Gardel theriot@st

1 03797126b6e6689cc02878e641e5271d3b9a5d3b Chun Id jjhchung@snu.ac.kr
1 03ca47766e6f3e8d5d71fa25d86803f82d690b38 Sun nan gbsun@implad.ac.cn
2 03ca47766e6f3e8d5d71fa25d86803f82d690b38 Xiao-Bo Sun gbsun@implad.ac.cn
2 048b240dcabbd623a6cda8c4236ea50d7961315c Vincent Nijman vnijman@brookes.ac.uk
2 04abbb8d91bd1a15a6c70ce3c0bd339f92468265 Charlanne Burke cburke@rockfound.org
2 04bc03c90437934a75fc6fdc228817234ef84c3a Zhenhua Dai zdai2009&#x00040
2 04d638b3c1d8b8fcd771e569e439b364d4ac2c1a Ty Hang hangvtt@oucru.org
1 04d638b3c1d8b8fcd771e569e439b364d4ac2c1a Han Ny hangvtt@oucru.org
3 05257a2230897ea006b3f68dbf0d71e1e7216f55 Paul Schnitzler Paul_Schnitzler@med.uni-heidelberg.de
3 0542a6ae26f57a5ffa35dcfebdec5004ff45688c Ruth Lynfield Ruth.Lynfield@state.mn.us
2 05774f9508a8c3399ffb20e758f933ffadc40fbd Ioannis Nezis I.Nezis@warwick.ac.uk
1 058b0cc7cecd43d80ed47196cb5376fa10efb58a Miriam Cebey-López Miriam.cebey.lopez@sergas.es
2 05a94d5f5083ff58320c659cd519536d3e6aa4a3 H Wodrich harald.wodrich

...

3 eaf0b485f290fa884dd18de71be79d86de20eb31 Lu Liang lianglu1@foxmail.com
3 eaf0b485f290fa884dd18de71be79d86de20eb31 Mengcen Qian qianmengcen@fudan.edu.cn
3 eaf0b485f290fa884dd18de71be79d86de20eb31 Yuxia Liang scudxliangyuxia@163.com
2 eaf0b485f290fa884dd18de71be79d86de20eb31 Juanjuan Zhang zhangjuan4486@163.com
3 eb8ac60527db35b10881cb4fd86b8a6e21983d02 Hongzhou Lu luhongzhou@fudan.edu.cn
3 eb8ac60527db35b10881cb4fd86b8a6e21983d02 Jingwen Ai jingwenai1990@126.com
3 eb8ac60527db35b10881cb4fd86b8a6e21983d02 Yinzhong Shen shenyinzhong@shphc.org.cn
3 eb8ac60527db35b10881cb4fd86b8a6e21983d02 Yang Li lixiangyang820@hotmail.com
3 eb8ac60527db35b10881cb4fd86b8a6e21983d02 Tao Li litaokc@126.com
3 eb8ac60527db35b10881cb4fd86b8a6e21983d02 Xian Zhou zhouxian-13@163.com
1 eb8ac60527db35b10881cb4fd86b8a6e21983d02 Zhang nan wenhongzhang_hs@126.com
2 eb8ac60527db35b10881cb4fd86b8a6e21983d02 ，qiran Zhang wenhongzhang_hs@126.com
2 eb8ac60527db35b10881cb4fd86b8a6e21983d02 ，yun Ling lingyun@shaphc.org
3 e

2 f339a22cebc1bc0099beb7aea41be09cc916e5dc François Ferron francois.ferron@afmb.univ-mrs.fr
3 f33c6d94b0efaa198f8f3f20e644625fa3fe10d2 Corey Watson corey.watson@mssm.edu
3 f40eaec892778166e4f8f8eeb576893e82a591be Yuzhen Zhang zhangyuzhen@suda.edu.cn
1 f40eaec892778166e4f8f8eeb576893e82a591be Bin Jiang jbin@suda.edu.cn
3 f47af9fe364fd9bac5e061142208d801667c5aca Zhaowei Zhu zhuzhaowei@csu.edu.cn
2 f47af9fe364fd9bac5e061142208d801667c5aca Jianjun Tang tangliang1226@126.com
3 f47af9fe364fd9bac5e061142208d801667c5aca Zhenfei Fang zhenfeifang@sina.com
3 f47af9fe364fd9bac5e061142208d801667c5aca Qiming Liu qimingliu@126.com
3 f47af9fe364fd9bac5e061142208d801667c5aca Xinqun Hu huxinqungs@163.com
3 f47af9fe364fd9bac5e061142208d801667c5aca Danyan Xu xudanyan02@sina.com
3 f47af9fe364fd9bac5e061142208d801667c5aca Jia He hejia2007@hotmail.com
3 f47af9fe364fd9bac5e061142208d801667c5aca Liang Tang tangliang1226@126.com
2 f47af9fe364fd9bac5e061142208d801667c5aca Shi Tai samtai2012@163.com
3 f49a9b36227

3 f9c6a656d0ce352cc1fa0df8d83e6d4a1f53d7c2 Nick Wilson nick.wilson@otago.ac.nz
3 f9c6a656d0ce352cc1fa0df8d83e6d4a1f53d7c2 Amanda Kvalsvig amanda.kvalsvig@otago.ac.nz
3 f9c6a656d0ce352cc1fa0df8d83e6d4a1f53d7c2 Lucy Barnard lucy.telfar-barnard@otago.ac.nz
3 f9c6a656d0ce352cc1fa0df8d83e6d4a1f53d7c2 Michael Baker michael.baker@otago.ac.nz
3 fbc41a8e025cd6eb2d7c156aad3b6923af2349c4 Chao Wu wuchao1984@zju.edu.cn
2 fbc41a8e025cd6eb2d7c156aad3b6923af2349c4 Shufa Zheng minzheng@zju.edu.cn
3 fbc41a8e025cd6eb2d7c156aad3b6923af2349c4 Yu Chen chenyuzy@zju.edu.cn
3 fbc41a8e025cd6eb2d7c156aad3b6923af2349c4 Min Zheng minzheng@zju.edu.cn
2 fbda9162de4c4a881b118dd140f1e976902df15e Qi-Chuan Zhang zhang_yonghai@126.com
2 fbebe4b66073c44cface2e842754bce26e3e2913 Xumao Zhao zhaoxm@lzu.edu.cn
2 fbebe4b66073c44cface2e842754bce26e3e2913 Yuehua Sun sunyh@ioz.ac.cn
2 fc15f433048e4d9464964930c36a1e7057ba1088 Jason Mclellan jmclellan@austin.utexas.edu
3 fc7a6b5d1852c5ecce2d20fd0d73d5f957ed7055 Benjamin Maier benja

2 0ed539fa6dbf35a97bad42eea7182ca5c36c47af Brenda Ang kwang@tll.org.sg
2 0ed539fa6dbf35a97bad42eea7182ca5c36c47af Jimmy Kwang kwang@tll.org.sg
2 0ed539fa6dbf35a97bad42eea7182ca5c36c47af J Kwang kwang@tll.org.sg
2 0f26fcb654f47c611134190653ed92fd38dd5d67 Hongmei Jiao jiao@yzu.edu.cn
2 0f26fcb654f47c611134190653ed92fd38dd5d67 Xinan Jiao jiao@yzu.edu.cn
2 0fc42c4b3227cbd5f17a24e441c9526435bc504a Fang Yuan fyuan@arcbs.redcross.org.au
2 0fcae4067d557b85dd27c6982344b763263e6ab2 Tim Mackey tmackey@ucsd.edu
2 1027702151e441039ec492e11d4fd20eb7da65dc William Lindsley wlindsley@cdc.gov
2 106bb31b6de9ffdd9f66285134e29df38c6957ab Barry Fields bfields@cdc.gov
3 11632bbc30436471453dedcc618a2efce7622e7e Diego Lara diego.lara@ochsner.org
2 139e6ba9edaf8b1518f33f6d1e2f134c4e2cd74e Eduardo Davila edavila@som.umaryland.edu
1 14ab6eeafd7fdbca309142856b179fe0438fd61a Yijing Li prof.lyijing@yahoo.com.cn
2 15725f58fc5a9f23c2d12dd7695557771f2dd857 Hui Tey jeannie_tey@moh.gov.sg
3 1a6a8f6d19437c3bcfcdce9557131

1 a866478edd938372f5321371609432b4eca3ebc9 Erica Saphire erica@scripps.edu
2 acfb5f151553011b81df2078523c2322b646cc58 Jeffrey Lee jlee@scripps.edu
2 ada90e72a1506a09225f70eeb53ef82c10bf2f65 Thomas Lane tom.lane@utah.path.edu
4 ae267a98af103856a22932cac54e9208b34da846 J Gavora helene.hayes@inra.fr
2 aea77148a52e6ebafa7162a57b19805da03a447c Ziad Memish zmemish@yahoo.com
2 af4d56adf262ed89e8005d2e78e3760370fa346e Susan Baker sbaker1@luc.edu
2 b1555b131850b0806f304b2005ad9e537a4ed7e8 Shan Sun dssun@mail.tcu.edu.tw
2 b1555b131850b0806f304b2005ad9e537a4ed7e8 Der-Shan Sun dssun@mail.tcu.edu.tw
3 b2a29d12403e49b5b2b560ce350632a886ed8a6e Gergely Tekes gergely.tekes@vetmed.uni-giessen.de
3 b75374ebf027be698e5ce4e721a4840dc4384240 Guido Antonelli guido.antonelli@uniroma1.it
2 bbe2e49fac5f007f3313b35f9b264bb4219aa67d Kentaro Kato kkato(at)obihiro.ac.jp
2 bbff324ce85ca876d0c96244b3c9fa479c7f300f Chris Bunce c.m.bunce@bham.ac.uk
3 bcad0a2902802ee1d5a8389d8bbbbcc61bb277ee John Ziebuhr john.ziebuhr@vi

In [168]:
# Remove conflicts (resolve to highest strength match)
for paper in authors.sha.unique():
    counts=(authors[authors.sha==paper].email.value_counts())
    
    cx=counts.index
    for c in range(0,len(counts)):
        #print(c.email)
        if(counts[c]>1):
            therecords=authors[(authors.sha==paper) & (authors.email==cx[c])]
            maxstrength=therecords.strengths.max()

            ids=therecords[therecords.strengths!=maxstrength].index
            authors.email[ids]=""

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]


In [169]:
# Apply cleaning to the emails field

# Find the most common domain endings in the set.
# Since majority are entries, we can use thse to help us 
# split concatenated strings. Good domains have >1 characrter.
Emails=authors.email[authors.notna().email]
CommonEndings={}
for em in Emails:
    Ending=em.split('.')[-1]
    if(len(Ending)>1):
        if(not Ending in CommonEndings.keys()):
            CommonEndings[Ending]=1
        else:
            CommonEndings[Ending]+=1

# An ending is popular (and so probably real)
# if it has >5 use cases in the set.           
PopularEndings=[]
PopularCounts=[]
for key in CommonEndings.keys():
    if(CommonEndings[key]>5):
        PopularEndings.append("."+key)
        PopularCounts.append(CommonEndings[key])
endings=np.array(PopularEndings)

# I noticed from the dataset that we need to protect these two, 
# since they contain within them other good (more popular) domain
# strins
endings=endings[np.argsort(PopularCounts)][::-1]
endings=np.concatenate([[".int",".gov.uk"],endings])

print("Popular endings:")
print(endings)

# This function fixes two pathologies:
#  1) Bad characters or strings jammed into the email address - common ones
#       picked out by observation of the list;
#
#  2) Valid email concatenated at end with more text, identified via
#       noting occurence of popular domain name in the string.
#
#   [We cannot fix concatenated at beginning, since string jammed on front is
#   still a valid formatted address, in most cases.]

def FixIt(em):
    # Forbidden word /symbol cleaning
    if(pd.isna(em)):
        return np.NaN
    forbidden=['author','emailaddress','correspondingauthor','telephone','<','>','*-','*',':','†']
    Changed=False
    for forb in forbidden:
        if(forb in em):
            splits=em.split(forb)
            for s in splits:
                #after split, the part contianing 
                #  an @ is an email address
                if "@" in s:   
                    em=s
                    
    # popular domain name based cleaning        
    for ending in endings:
        found=(em.find(ending))
        wheresat=em.find("@")

        if((found>0) and ((found+len(ending))<=len(em)) and (found > wheresat)):
            em=em[0:found+len(ending)]
            break
    
    return em



Popular endings:
['.int' '.gov.uk' '.com' '.edu' '.cn' '.uk' '.de' '.fr' '.org' '.ca' '.au'
 '.gov' '.hk' '.tw' '.jp' '.kr' '.nl' '.it' '.ch' '.sg' '.br' '.es' '.se'
 '.be' '.net' '.il' '.sa' '.nz' '.fi' '.pl' '.za' '.in' '.at' '.no' '.eu'
 '.mil' '.ir' '.pt' '.ie' '.ru' '.dk' '.gr' '.cat' '.int' '.my' '.th'
 '.cz' '.ar' '.tr' '.mx' '.qa' '.cl' '.co' '.to' '.flzhang&#x00040' '.hu'
 '.si' '.wang' '.com†']


In [170]:

# Sample 100 for smell test
for em in Emails[0:100]:
    em2=FixIt(em)
    if(em2!=em):
        print(em + " -> " + em2)


NewEmails=[]        
for em in authors.email:
    NewEmails.append(FixIt(em))
        
#Now go ahead and apply it to the dataset
#def FixEmail(row):
#    newem=FixIt(row.email)
#    if(newem!=row.email):
#        row.email=newem
#    return row

authors.email=NewEmails

milne@csse.uwa.edu.au -> milne@csse.uwa.edu
e.:ehondo@agr.nagoya-u.ac.jp -> ehondo@agr.nagoya-u.ac.jp
marin@fcien.edu.uym.marín. -> marin@fcien.edu
mmiceli@med.umich.edu. -> mmiceli@med.umich.edu
craig.mccormick@dal.cac.m. -> craig.mccormick@dal.ca
d.khaperskyy@dal.cad.a.k. -> d.khaperskyy@dal.ca
marta.gaglia@tufts.edum.m.g. -> marta.gaglia@tufts.edu
niuyuming@yeah.net. -> niuyuming@yeah.net
claudio.canal@ufrgs.brc.w.canal. -> claudio.canal@ufrgs.br
yxiao@mail.xjtu.edu.cny.xiao -> yxiao@mail.xjtu.edu
rbaric@email.unc.edu. -> rbaric@email.unc.edu
chuakb@tll.org.sg -> chuakb@tll.org
jidoyaga@stanford.edu. -> jidoyaga@stanford.edu
nbellei@uol.com.br -> nbellei@uol.com
s.puechmaille@gmail.com. -> s.puechmaille@gmail.com
rbaric@email.unc.edu. -> rbaric@email.unc.edu
es:peter.durr@csiro.aup.a.durr -> peter.durr@csiro.au
linfa.wang@duke-nus.edu.sg -> linfa.wang@duke-nus.edu
evelyn_koay@nuhs.edu.sg -> evelyn_koay@nuhs.edu
evelyn_koay@nuhs.edu.sg -> evelyn_koay@nuhs.edu
pfeffer@ibmc-cnrs.unistr

In [171]:
# Save the output file
authors.drop('strengths',axis=1)
authors.to_csv("AuthorsWithEmails.csv")

In [174]:
sum(authors.notna().email)

15678